# Words in The Office 

Data Source: Redditor/misunderstoodpoetry, https://docs.google.com/spreadsheets/d/18wS5AAwOh8QO95RwHLS95POmSNKA2jjzdt0phrxeAE0/edit#gid=747974534 


### Counting the Word Frequency


In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from collections import Counter
import os

current_directory = os.getcwd()

# nltk resources
nltk.download('punkt')
nltk.download('stopwords')

# read in data and drop empty lines
df = pd.read_csv('/the-office-lines-text.csv')
df = df.dropna(subset=['line_text'])

# english stop words
stop_words = set(stopwords.words('english'))

# tokenize and remove the stopwords
df['tokens'] = df['line_text'].apply(lambda x: [word for word in word_tokenize(x.lower()) if word not in stop_words and word.isalpha()])

# create a single list of words
all_tokens = []
for tokens in df['tokens']:
    all_tokens.extend(tokens)

# count the word frequency
word_freq = Counter(all_tokens)

# create data frame and sort
word_freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)

# define the new file path and create the csv
file_path = os.path.join(current_directory, 'word_frequency.csv')
word_freq_df.to_csv(file_path, index=False)

print(f"Completed: CSV filed created at {file_path}")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/holliegardner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/holliegardner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Word frequency CSV file has been created at /Users/holliegardner/Desktop/the_office_project/word_frequency.csv


### Grouping Word Frequency By Character for Word Clouds

In [4]:
from collections import Counter, defaultdict  

# function for counting words by each speacker
def count_words(group):
    all_words = []
    for words in group['tokens']:
        all_words.extend(words)
    return Counter(all_words)

# count the words for each character
word_frequency_by_speaker = df.groupby('speaker').apply(count_words)

# creating dictionary to hold word counts for each character
word_dict = defaultdict(lambda: defaultdict(int))

for speaker, counter in word_frequency_by_speaker.items():
    for word, count in counter.items():
        word_dict[word][speaker] += count

# converting dictionary to data frame
word_freq_df = pd.DataFrame.from_dict(word_dict, orient='index').fillna(0)

# take sum of word counts and filter to speakers with greater than 1,500 words to avoid 700+ character rows
speaker_word_totals = word_freq_df.sum(axis=0)
filtered_speakers = speaker_word_totals[speaker_word_totals >= 1500].index

# Filter the DataFrame to include only these speakers
word_freq_df = word_freq_df[filtered_speakers]

# export to csv
word_freq_df.to_csv('character_word_frequency_filtered.csv')


Filtered character word frequency CSV file has been created.
